In [ ]:
!pip install noisereduce

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
import imageio
from IPython.display import Image
import matplotlib.image as mpimg
#MUSIC PROCESS
import pydub
from scipy.io.wavfile import read, write
import librosa
import librosa.display
import IPython
from IPython.display import Audio
import noisereduce as nr
import scipy
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape, Conv2DTranspose, LeakyReLU
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

In [ ]:
first_wav = "../input/piano-triads-wavset/piano_triads/A_maj_2.wav"
second_wav = "../input/piano-triads-wavset/piano_triads/A_min_5.wav"
third_wav = "../input/piano-triads-wavset/piano_triads/Bb_maj_7.wav"

In [ ]:
first_fav,first_temp = read(first_wav)
second_fav,second_temp = read(second_wav)
third_fav,third_temp = read(third_wav)

In [ ]:
general_rate = first_fav

In [ ]:
Audio(first_temp, rate=general_rate)

In [ ]:
Audio(second_temp, rate=general_rate)

In [ ]:
Audio(third_temp, rate=general_rate)

In [ ]:
first_temp_array = np.array(first_temp)
second_temp_array = np.array(second_temp)
third_temp_array = np.array(third_temp)

In [ ]:
print(first_temp_array[0])

In [ ]:
print(second_temp_array[0])

In [ ]:
print(third_temp_array[0])

In [ ]:
print(first_temp_array)
print(first_temp_array.shape)
print(first_temp_array.dtype)

In [ ]:
print(second_temp_array)
print(second_temp_array.shape)
print(second_temp_array.dtype)

In [ ]:
print(third_temp_array)
print(third_temp_array.shape)
print(third_temp_array.dtype)

In [ ]:
concat_wav = np.concatenate([first_temp_array,second_temp_array,third_temp_array])

In [ ]:
print(concat_wav)
print(concat_wav.shape)
print(concat_wav.dtype)

In [ ]:
Audio(concat_wav, rate=general_rate)

In [ ]:
Main_Path = Path("../input/piano-triads-wavset/piano_triads")

In [ ]:
Path_List = list(Main_Path.glob(r"*.wav"))

In [ ]:
Total_Wav_List = []

for music_wav in Path_List:
    rate_pick,temp_pick = read(music_wav)
    Total_Wav_List.append(temp_pick)

In [ ]:
print(Total_Wav_List[0:10])

In [ ]:
concat_list = np.concatenate([Total_Wav_List])

In [ ]:
print(concat_list)
print(concat_list.shape)
print(concat_list.dtype)

In [ ]:
Audio(concat_list[0], rate=44100)

In [ ]:
print(concat_list[0].shape)
print(concat_list[0].dtype)

In [ ]:
Reshape_Concat = concat_list.reshape(144,132300,1,1)

In [ ]:
print(Reshape_Concat.shape)
print(Reshape_Concat.dtype)

In [ ]:
X_Train = Reshape_Concat
X_Train = X_Train / 255.

In [ ]:
print(X_Train.shape)
print(X_Train.dtype)

In [ ]:
discriminator = Sequential()
discriminator.add(Flatten(input_shape=[132300,]))
discriminator.add(Dense(50,activation="relu"))
discriminator.add(Dense(1,activation="sigmoid"))

In [ ]:
discriminator.compile(loss="binary_crossentropy",optimizer="rmsprop")

In [ ]:
print(discriminator.summary())

In [ ]:
discriminator.trainable = False

In [ ]:
generator = Sequential()
generator.add(Dense(50,activation="relu",input_shape=[132300]))
generator.add(Dense(132300,activation="relu"))
generator.add(Reshape([132300,1]))

In [ ]:
print(generator.summary())

In [ ]:
GAN = Sequential([generator,discriminator])

In [ ]:
print(GAN.summary())

In [ ]:
GAN.compile(loss="binary_crossentropy",optimizer="rmsprop")

In [ ]:
start_period = 0
batch_size = 1
dim_size = 132300
iterations = 50

Fake_List = []

In [ ]:
generator,discriminator = GAN.layers

for epoch in range(iterations):
    print(f"Current on {epoch+1}")
    i = 0
    for x_batch in X_Train:
        
        i +=1
       
        print(f"Currently on batch {i} of {len(Total_Wav_List)//batch_size}")


        noise =  np.random.normal(size=(batch_size,dim_size))
        gen_mus = generator(noise)
        
        stop = start_period + batch_size
        
        real_mus = X_Train[start_period:stop]
        real_mus = real_mus.reshape(1,real_mus.shape[1],1)

        x_fake_or_real = np.concatenate([gen_mus,real_mus])

        y1 = tf.constant([[0.0]]*batch_size + [[1.0]]*batch_size)

        discriminator.trainable = True
        discriminator.train_on_batch(x_fake_or_real,y1)



        noise = tf.random.normal(shape=[batch_size,dim_size])
        y2 = tf.constant([[1.0]]*batch_size)
        discriminator.trainable = False
        GAN.train_on_batch(noise,y2)
        
        
        Fake_List.append(gen_mus)

In [ ]:
print(Fake_List[19].shape)

In [ ]:
F01_Wav_M = Fake_List[6]
F0_Wav_M = Fake_List[7]
F1_Wav_M = Fake_List[8]
S2_Wav_M = Fake_List[9]
T3_Wav_M = Fake_List[10]
FR4_Wav_M = Fake_List[11]
FV5_Wav_M = Fake_List[12]
SX6_Wav_M = Fake_List[13]

In [ ]:
Reshape_Wav_M = tf.reshape(Fake_List[4], [132300,])

In [ ]:
print(Reshape_Wav_M.shape)

In [ ]:
Audio(Reshape_Wav_M, rate=general_rate)

In [ ]:
Mixing_Wav = np.concatenate([F01_Wav_M,F0_Wav_M,F1_Wav_M,S2_Wav_M,T3_Wav_M,FR4_Wav_M,FV5_Wav_M,SX6_Wav_M])

In [ ]:
print(Mixing_Wav.shape)

In [ ]:
Mix_Wav_M = tf.reshape(Mixing_Wav, [8,132300])

In [ ]:
print(Mix_Wav_M.shape)

In [ ]:
Audio(Mix_Wav_M, rate=general_rate)

In [ ]:
ix = 0
for music_wav in Mix_Wav_M:
    ix +=1
    scipy.io.wavfile.write(f"Mix{ix}.wav",general_rate,music_wav)

In [ ]:
Mix_Wav_M = np.array(Mix_Wav_M)

In [ ]:
Pre_Nu_M = Mix_Wav_M[0]

In [ ]:
print(Mix_Wav_M[0].shape)

In [ ]:
noisy_part = Mix_Wav_M[0][0:132300]

In [ ]:
reduced_noise = nr.reduce_noise(audio_clip=Pre_Nu_M, noise_clip=noisy_part, verbose=True,
                                prop_decrease=True,use_tensorflow=False,n_std_thresh=True,pad_clipping=True)

In [ ]:
Audio(reduced_noise, rate=general_rate)

In [ ]:
scipy.io.wavfile.write("MixR.wav",general_rate,reduced_noise)

In [ ]:
discriminator_N = Sequential()
discriminator_N.add(Flatten(input_shape=[132300,]))
discriminator_N.add(Dense(50,activation="relu"))
discriminator_N.add(Dense(1,activation="sigmoid"))

In [ ]:
discriminator_N.compile(loss="binary_crossentropy",optimizer=RMSprop(lr=0.0001,clipvalue=1.0,decay=1e-8))

In [ ]:
discriminator_N.trainable = False

In [ ]:
generator_N = Sequential()
generator_N.add(Dense(50,activation="relu",input_shape=[132300]))
generator_N.add(Dense(132300,activation="tanh"))
generator_N.add(Reshape([132300,1]))

In [ ]:
GAN_N = Sequential([generator,discriminator])

In [ ]:
GAN_N.compile(optimizer=RMSprop(lr=0.0001,clipvalue=1.0,decay=1e-8),loss="binary_crossentropy")

In [ ]:
start_period = 0
batch_size = 1
dim_size = 132300
iterations = 1000

Gan_M_Fake_List = []

In [ ]:
for step in range(iterations):
    random_noising = np.random.normal(size=(batch_size,dim_size))
    generator_Mus = generator_N.predict(random_noising)
    
    stop = start_period + batch_size
    
    real_mus = X_Train[start_period:stop]
    real_mus = real_mus.reshape(1,real_mus.shape[1],1)
    
    Combinated_IMG = np.concatenate([gen_mus,real_mus])
    
    labels = np.concatenate([np.ones((batch_size,1)),np.zeros((batch_size,1))])
    labels = labels + 0.5 * np.random.random(labels.shape)
    
    Discriminator_LS = discriminator_N.train_on_batch(Combinated_IMG,labels)
    
    random_noising = np.random.normal(size=(batch_size,dim_size))
    
    Misleading_IMG = np.zeros((batch_size,1))
    
    Adversarial_LS = GAN_N.train_on_batch(random_noising,Misleading_IMG)
    
    start_period = start_period + batch_size
    
    if start_period > len(X_Train) - batch_size:
        start_period = 0
        
    
        
    print("DISCRIMINATOR LOSS: ", Discriminator_LS)
    print("ADVERSARIAL LOSS: ", Adversarial_LS)
    Gan_M_Fake_List.append(generator_Mus)

In [ ]:
General_Pre = Gan_M_Fake_List[899]

In [ ]:
F_Wav_M = Gan_M_Fake_List[0]
S2_Wav_M = Gan_M_Fake_List[1]
T_Wav_M = Gan_M_Fake_List[2]
FR_Wav_M = Gan_M_Fake_List[3]
FV_Wav_M = Gan_M_Fake_List[4]
SX_Wav_M = Gan_M_Fake_List[5]

In [ ]:
Reshape_WavF_M = tf.reshape(FV_Wav_M, [132300,])

In [ ]:
print(Reshape_WavF_M.shape)

In [ ]:
Audio(Reshape_WavF_M, rate=general_rate)

In [ ]:
Predict_Noise = np.random.normal(size=(30,dim_size))

In [ ]:
Gen_Predict_N = generator_N(Predict_Noise)

In [ ]:
MixGen_Wav_M = tf.reshape(Gen_Predict_N, [30,132300])

In [ ]:
print(MixGen_Wav_M.shape)

In [ ]:
Audio(MixGen_Wav_M[9], rate=general_rate)